In [1]:
# 필수 라이브러리 임포트
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

# 데이터 로드
housing = fetch_california_housing()
X, y = housing.data, housing.target

# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [2]:
# 하이퍼파라미터 범위
alpha_range = np.logspace(-3, 1, 50)

# 파라미터 그리드 생성
param_grid = {'alpha': alpha_range}

# 모델 선언
lasso = Lasso()
ridge  = Ridge()
elasticnet = ElasticNet()

In [6]:
lasso_cv = GridSearchCV(lasso, param_grid, cv=5, scoring='r2', n_jobs=-1)
lasso_cv.fit(X_train, y_train)
print(f"Lasso Best Alpha: {lasso_cv.best_params_['alpha']}")
print(f"Lasso Best R2: {lasso_cv.best_score_}")


Lasso Best Alpha: 0.001
Lasso Best R2: 0.6114960326349894


In [7]:
ridge_cv = GridSearchCV(ridge, param_grid, cv=5, scoring='r2', n_jobs=-1)
ridge_cv.fit(X_train, y_train)
print(f"Ridge Best Alpha: {ridge_cv.best_params_['alpha']}")
print(f"Ridge Best R2: {ridge_cv.best_score_}")


Ridge Best Alpha: 0.49417133613238334
Ridge Best R2: 0.6114840258561682


In [8]:
elasticnet_cv = GridSearchCV(elasticnet, param_grid, cv=5, scoring='r2', n_jobs=-1)
elasticnet_cv.fit(X_train, y_train)
print(f"ElasticNet Best Alpha: {elasticnet_cv.best_params_['alpha']}")
print(f"ElasticNet Best R2: {elasticnet_cv.best_score_}")


ElasticNet Best Alpha: 0.001
ElasticNet Best R2: 0.6114860265335377


In [9]:
# 최적 모델 적용
best_lasso = Lasso(alpha=lasso_cv.best_params_['alpha']).fit(X_train, y_train)
best_ridge = Ridge(alpha=ridge_cv.best_params_['alpha']).fit(X_train, y_train)
best_elasticnet = ElasticNet(alpha=elasticnet_cv.best_params_['alpha']).fit(X_train, y_train)

# 예측
y_pred_lasso = best_lasso.predict(X_test)
y_pred_ridge = best_ridge.predict(X_test)
y_pred_elasticnet = best_elasticnet.predict(X_test)

# R² 계산
r2_lasso = r2_score(y_test, y_pred_lasso)
r2_ridge = r2_score(y_test, y_pred_ridge)
r2_elasticnet = r2_score(y_test, y_pred_elasticnet)

print(f"Test R2 (Lasso): {r2_lasso}")
print(f"Test R2 (Ridge): {r2_ridge}")
print(f"Test R2 (ElasticNet): {r2_elasticnet}")


Test R2 (Lasso): 0.5769212309657656
Test R2 (Ridge): 0.575802965737464
Test R2 (ElasticNet): 0.5765975290718771


하이퍼파라미터 범위
np.logspace(-3, 1, 50) : 10의 -3승에서 1승까지의 로그 스케일로 50개의 값을 생성(로그스케일 사용하는 이유 = 규제 강도(alpha)가 지수적으로 변할 때 성능 변화를 효과적으로 탐색)

param_grid : {'alpha' : alpha_range} = 그리드 서치에 사용하기 위한 하이퍼파라미터 후보

순회하면서 최적의 하이퍼파라미터 찾기위해 이렇게함.

GridSearchCV
GridSearchCv(모델(ex.ridge), param_grid(alpha값), cv=5(:5-Fold 교차 검증 사용), scoring='r2'(:모델 성능 평가 기준을 R2 점수로 설정), n-jobs=-1(:모든 cpu 코어 사용하여 병렬 처리))

ridge_cv.best_params_['alpha'] : 릿지 모델의 최적의 알파 값

지금까지 흐름 정리

선형/비선형 데이터 표현은 부정확.
데이터가 선형/비선형 모델에 적합하다 표현이 맞음.

보통의 json처럼 구조적 데이터의 경우 선형 모델이 대부분 적합함.
이미지, 오디오, 영상같은 복잡하거나 다차원의 경우 비선형 모델이 적합함.

선형/비선형 모델 적합성을 어떻게 판단?
데이터를 보면 알 수 있는 경우:
경험과 도메인 지식이 있을 때, 데이터의 특성이 명확할 때

데이터를 직접 테스트해서 알아야 하는 경우:
복잡한 데이터나 비정형 데이터일 때
<!-- 데이터가 섞여 있을 때 -->

여부를 확신할 수 없을 때
여러 모델 사용해서 비교

그리드서치가 중요한듯 랏소,릿지,엘라스틱넷같은 모델의 최적의 하이퍼파라미터를 찾을 수 있게해줌.

최적 모델 선택 방법:
모델 학습:
 선형이든 비선형모델이든 테스트

성능 비교:
 각 모델의 r2점수와 오차(mse)를 비교
 가장 성능이 좋은 모델을 최종 선택

결론 도출:
 선형 모델 성능이 우수하면 선형 모델이 적합
 비면 비